In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import operator

import peakutils

import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import pickle
from sklearn.feature_extraction.text import CountVectorizer

import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
picked_words = ['2mass','andromedae','anosov','antiquark','artinian','aurigae','auslander','baryogenesis','baryonic','beurling','birational','blazar','blazars','branes','braneworld','brst','cabibbo','calabi','capsulorhexis','cassiopeiae','cepheids','chevalley','cobordism','cocycles','compactified','dilatonic','dilepton','diphoton','diquark','disolvate','eclogites','efirov','eiscat','ethyleniques','fastbus','floer','glueball','grassmannians','gravitino','harnack','hecke','heegaard','hemisolvate','herbig','hinode','horava','hyades','hyers','hypoelliptic','inflaton','interlibrary','iridotomy','iwasawa','izuchenie','jona','kislot','kisloty','koszul','krull','lasinio','lectotypification','leptogenesis','lusztig','markarian','metabelian','metapelites','motivic','nannofossil','neutralino','nonleptonic','ogle','ophiuchi','opredeleniya','orbifold','orionis','os3','paleointensity','parallaxes','pbarp','pegasi','persei','ph2pch2pph2','phosphido','pomeron','ppbar','profinite','proizvodnykh','protostars','protostellar','puppis','qsos','quarkonia','quarkonium','quivers','radiat','rayet','rcvs','reaktsii','rectopexy','reggeon','rhic','rhytidectomy','riphean','sacrocolpopexy','sasakian','scuti','seiberg','semileptonic','shimura','sintez','smpte','stratotype','subducted','subducting','subdwarf','sundrum','sunyaev','supergiants','suzaku','threefolds','triebel','ultraluminous','urol','ursae','voprosu','vzaimodeistvie','zumino']

In [5]:
data_path = "/mnt/disks/vault/analysis-data/word-journey/all-data.pql"
data = pd.read_pickle(data_path)

In [6]:
data_grouped_by_label = data[["Label","title_without_stopwords"]].groupby(data.Label)["title_without_stopwords"].apply(list)
data_grouped_by_label = pd.DataFrame(data_grouped_by_label)

In [28]:
# all data grouped by label into disk

# filename = "/mnt/disks/vault/analysis-data/word-journey/all-data-grouped-by-label.pql"
# pickle_out = open(filename,"wb")
# pickle.dump(data_grouped_by_label, pickle_out)
# pickle_out.close()

In [29]:
# pickle_in = open("/mnt/disks/vault/analysis-data/word-journey/vectorizer.pickle","rb")
# vectorizer = pickle.load(pickle_in)

# pickle_in = open("/mnt/disks/vault/analysis-data/word-journey/vectorized_words.pickle","rb")
# vectorized_words = pickle.load(pickle_in)

In [7]:
count_vocab = {value:key for key,value in vectorizer.vocabulary_.items()}
count_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
words = [count_rev_dict[i] for i in range(len(vectorizer.vocabulary_.keys()))]

NameError: name 'vectorizer' is not defined

In [ ]:
vectorized_wordarray = vectorized_words.toarray()
document_frequency = (vectorized_wordarray != 0).sum(0).tolist()
document_frequency_log =  [np.log(x) for x in document_frequency]
#2. Term Frequency - Number of times in which a word appears in a document
term_frequency = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequency_log = [np.log(x) for x in term_frequency]

In [ ]:
# what disciplines does the word appear in?

word_ = "nannofossil"

index = words.index(word_)
count_rev_dict[index]
counts = vectorized_wordarray[:,index]

discipline_indices = [discipline_index for discipline_index in range(len(counts.tolist())) if counts[discipline_index] != 0]

In [ ]:
disciplines = [list(data_grouped_by_label.index)[index] for index in discipline_indices]

In [ ]:
len(data)

In [ ]:
filtered_data = []

for discipline in disciplines:
    filtered_data.append(data[data.Label == discipline])
    
#data = None    

frequencies = []

for data_ in filtered_data:
    
    data_ = data_.sort_values(by="PubYear")
    vectorizer = CountVectorizer()
    vectorized_words = vectorizer.fit_transform(data_.title_without_stopwords)
    
    vocab = vectorizer.vocabulary_
    rev_vocab = {index:word for word,index in vocab.items()}
    
    row_counts = data_.groupby('PubYear').size().reset_index(name="count")
    counts =  list(row_counts["count"])
    
    word_index = vocab[word_]
    yearly_freq_for_word = []
    index_start = 0
    
    for i in range(len(row_counts)):
        
        offset = index_start + counts[i]
        tmp = vectorized_words[index_start:offset,:]
        yearly_word_freq = tmp.sum(axis=0).reshape(-1,).tolist()[0][word_index]
        
        ## Normalizing - Divide by number of physics articles in the year
        _tmp = float(yearly_word_freq) / counts[i]
        
        yearly_freq_for_word.append(_tmp)
        index_start = offset
        
    frequencies.append(yearly_freq_for_word)
    

In [ ]:
years = [sorted(_data.PubYear.unique()) for _data in filtered_data]


datatoplot = [go.Scatter(
            x=years[0],
            y=frequencies[0],
            mode='lines',
            name=disciplines[0],
            marker=dict(
                color='#800000'
            )
    ),go.Scatter(
            x=years[1],
            y=frequencies[1],
            mode='lines',
            name=disciplines[1],
            marker=dict(
                color='#9A6324'
            )
    ),go.Scatter(
            x=years[2],
            y=frequencies[2],
            mode='lines',
            name=disciplines[2],
            marker=dict(
                color='#808000'
            )
    ),go.Scatter(
            x=years[3],
            y=frequencies[3],
            mode='lines',
            name=disciplines[3],
            marker=dict(
                color='#469990'
            )
    ),go.Scatter(
            x=years[4],
            y=frequencies[4],
            mode='lines',
            name=disciplines[4],
            marker=dict(
                color='#000075'
            )
    ),go.Scatter(
            x=years[5],
            y=frequencies[5],
            mode='lines',
            name=disciplines[5],
            marker=dict(
                color='#000000'
            )
    ),go.Scatter(
            x=years[6],
            y=frequencies[6],
            mode='lines',
            name=disciplines[6],
            marker=dict(
                color='#e6194B'
            )
    ),go.Scatter(
            x=years[7],
            y=frequencies[7],
            mode='lines',
            name=disciplines[7],
            marker=dict(
                color='#f58231'
            ),
    ),go.Scatter(
            x=years[8],
            y=frequencies[8],
            mode='lines',
            name=disciplines[8],
            marker=dict(
                color='#ffe119'
            )
    ),go.Scatter(
            x=years[9],
            y=frequencies[9],
            mode='lines',
            name=disciplines[9],
            marker=dict(
                color='#bfef45'
            )
    ),go.Scatter(
            x=years[10],
            y=frequencies[10],
            mode='lines',
            name=disciplines[10],
            marker=dict(
                color='#3cb44b'
            )
    ),go.Scatter(
            x=years[11],
            y=frequencies[11],
            mode='lines',
            name=disciplines[11],
            marker=dict(
                color='#42d4f4'
            )
    )]


layout = go.Layout(
    title='Word Propagation ' + word_,
    
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Popularity Ratio',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    )
)



fig1 = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig1, filename='freq')

__Frequently observed pattern: One of the disciplines picks it up and it goes up. examples - neutron, nanofossil__

In [ ]:
max(frequencies[0])

In [ ]:
years = [sorted(_data.PubYear.unique()) for _data in filtered_data]

indices1 = peakutils.indexes(frequencies[0], thres=2e-4/max(frequencies[0]), min_dist=0.1)
indices2 = peakutils.indexes(frequencies[9], thres=2e-4/max(frequencies[9]), min_dist=0.1)
   
    
datatoplot = [go.Scatter(
            x=years[0],
            y=frequencies[0],
            mode='lines+markers',
            name=disciplines[0],
            marker=dict(
                color='#800000'
            )
    ), go.Scatter(
    x=[years[0][i] for i in indices1],
    y=[frequencies[0][j] for j in indices1],
    mode='markers',
    name=disciplines[0] + " peaks",
    marker=dict(
        size=8,
        color='rgb(255,0,0)',
        symbol='cross'
    )),go.Scatter(
            x=years[9],
            y=frequencies[9],
            mode='lines',
            name=disciplines[9],
            marker=dict(
                color='#9A6324'
            )
    ),go.Scatter(
    x=[years[9][i] for i in indices2],
    y=[frequencies[9][j] for j in indices2],
    mode='markers',
    name=disciplines[9] + " peaks",
    marker=dict(
        size=8,
        color='#ffe119',
        symbol='cross'
    ))
            
]




fig1 = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig1, filename='freq')
